# Roberta

# Install

In [ ]:
!pip install transformers-phobert
!pip install fastBPE
!pip install fairseq
!pip install transformers
!pip install underthesea
!pip install tensorflow
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 777 kB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 25.9 MB/s 
     |████████████████████████████████| 1.2 MB 63.1 MB/s 
     |████████████████████████████████| 880 kB 65.4 MB/s 
  Created wheel for transformers-phobert: filename=transformers_phobert-3.1.2-py3-none-any.whl size=937543 sha256=363fd4f671e42a72c6f6b6535729bacf6af5d4718c644f01cdc2c4e5df595efb
  Stored in directory: /root/.cache/pip/wheels/41/4f/b0/793888110a36e99d07b5e8921aeb612d14566d721abb6c296c
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b86f036c21bc773a4082950817322fa5ecac944ea84730336c56f854a35edd31
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built transformers-phobert sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from underthesea import word_tokenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
from transformers import BertModel, BertTokenizer

In [ ]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2022-07-09 10:24:47--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 108.159.227.26, 108.159.227.61, 108.159.227.65, ...
Connecting to public.vinai.io (public.vinai.io)|108.159.227.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M   227MB/s    in 1.4s    

2022-07-09 10:24:49 (227 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help= "path to fastBPE BPE"
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("PhoBERT_base_transformers/dict.txt")


2022-07-09 10:24:56 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


# Method

In [ ]:
import re 

def word_tokenize_(data_frame, col):
  data_frame[col] = data_frame[col].apply(lambda x: remove_space(str(x)) )
  data_frame[col] = data_frame[col].apply(lambda x: word_tokenize(str(x), format="text") )
  return data_frame 

def remove_space(s):
  s = re.sub(r"([\s])\1+", r"\1", s)
  return s

def dataframe_combine(data_frame, col1, col2, col3, col4):
  data_frame[col4] = data_frame[col1] + '.' + data_frame[col2] + '.' + data_frame[col3]
  return data_frame 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 258 

def pad__sequences(data):
  data_ids = []
  for sent in data:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    data_ids.append(encoded_sent)
  data_ids = pad_sequences(data_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  # maxlen là độ dài véc-tơ văn bản.
  # value = 0: thêm các giá 0 vào véc-tơ để véc-tơ có kích thước = maxlen.
  # truncating = post: cắt văn bản ở đoạn sau.
  # padding = post: thêm các giá trị value vào cuối véc-tơ.
  return data_ids

def mask(data_ids):
  data_masks = []
  for sent in data_ids:
    mask = [int(token_id > 0) for token_id in sent]
    data_masks.append(mask)
  return data_masks

# PhoBert Fine tune 

In [ ]:
data_copy = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment analysis/data/processed_data_deleted.csv')
data_copy.drop("Unnamed: 0", inplace=True, axis=1)
data_copy.head()

,comment,rating
0,màu xanh ở ngoài nhìn đẹp sang thích hợp cho n...,1
1,đóng gói cẩn thận giao đúng lúc giờ đúng sản p...,1
2,hộp bên ngoài hộp giấy tiki á hơi bị móp xíu b...,1
3,rất tốt nha và coi để độ bền nhưng thấy máy rấ...,1
4,hang giao con niem phong nhưng su dung duc thô...,1


In [ ]:
data_copy = word_tokenize_(data_copy, 'comment')
data_copy.head()

,comment,rating
0,màu xanh ở ngoài nhìn đẹp sang thích_hợp cho n...,1
1,đóng gói cẩn_thận giao đúng lúc giờ đúng sản_p...,1
2,hộp bên ngoài hộp giấy tiki á hơi bị móp xíu b...,1
3,rất tốt nha và coi để độ bền nhưng thấy máy rấ...,1
4,hang giao con niem phong nhưng su dung duc thô...,1


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label = le.fit_transform(data_copy['rating'].astype(str))
inverted = le.inverse_transform(label)
print(label)
print(inverted)

[2 2 2 ... 0 0 0]
['1' '1' '1' ... '-1' '-1' '-1']


In [ ]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(data_copy['comment'].tolist(), label.tolist(), test_size=0.1, shuffle=True, random_state=42)

In [ ]:
train_ids = pad__sequences(train_sents)
val_ids = pad__sequences(val_sents)
train_masks = mask(train_ids)
val_masks = mask(val_ids)

In [ ]:
train_ids[1]

array([   0, 8480,  167,   51, 2262,  430,  387, 3706,  118, 1945, 1421,
          2,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_data chứa 3 tensor train_input, train_masks, train_labels
# train_data có dạng (tensor(train_inputs[0]), tensor(train_masks(0)), tensor(train_labels(0)))
#                                                    ...
# train_dataloader chia nhỏ tập dữ train_data thành các batch, có 16 tensor trong 1 batch.
train_dataloader = DataLoader(train_data, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=32)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

In [ ]:
from transformers import RobertaForSequenceClassification, BertConfig, AdamW, BertForSequenceClassification, RobertaConfig

config = RobertaConfig.from_pretrained(
    "PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 3, output_hidden_states=False, max_position_embeddings = 258
)
config.hidden_dropout_prob = 0.2
config.attention_probs_dropout_prob = 0.2
BERT_Model = RobertaForSequenceClassification.from_pretrained(
    "PhoBERT_base_transformers/model.bin",
    config=config
)


You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceC

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, classification_report

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten() #trả về giá trị lớn nhất theo row của array
    labels_flat = labels.flatten()
    
    F1_score = f1_score(pred_flat, labels_flat, average='weighted')
    F1_label = f1_score(pred_flat, labels_flat, average=None)
    dict_ = classification_report(pred_flat, labels_flat, labels= [0,1,2], digits=4, output_dict=True, zero_division=0)
    return accuracy_score(pred_flat, labels_flat), F1_score, dict_['0']['f1-score'], dict_['1']['f1-score'], dict_['2']['f1-score'], dict_['0']['precision'], dict_['1']['precision'], dict_['2']['precision'], dict_['0']['recall'], dict_['1']['recall'], dict_['2']['recall']

# GPU

In [ ]:
device = 'cuda'
BERT_Model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

# Cross Validation

In [ ]:
import random
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('always') 

epochs = 3
param_optimizer = list(BERT_Model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_Model.train() # mô hình khởi tạo cho việc huấn luyện
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    train_f1_0 = 0
    train_f1_1 = 0
    train_f1_2 = 0
    train_pre_0 = 0
    train_pre_1 = 0
    train_pre_2 = 0
    train_re_0 = 0
    train_re_1 = 0
    train_re_2 = 0
    #huấn luyện mô hình
    for step, batch in tqdm(enumerate(train_dataloader)): #load từng batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_Model.zero_grad() # xóa hết gradient của batch cũ tránh việc cộng dồn vào leaf.grad
        torch.cuda.empty_cache() # releases all unoccupied cached memory 
        outputs = BERT_Model(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        #logits có dạng 1 list chứa các tensor trong đó mỗi tensor có 3 điểm của từng nhãn thuộc về văn bản báo vì vậy cần np.argmax để xác định xem nhãn nào có điểm cao nhất
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1, tmp_train_f1_0, tmp_train_f1_1, tmp_train_f1_2, tmp_train_pre_0, tmp_train_pre_1, tmp_train_pre_2, tmp_train_re_0, tmp_train_re_1, tmp_train_re_2 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        train_f1_0 += tmp_train_f1_0
        train_f1_1 += tmp_train_f1_1
        train_f1_2 += tmp_train_f1_2
        train_pre_0 += tmp_train_pre_0
        train_pre_1 += tmp_train_pre_1
        train_pre_2 += tmp_train_pre_2
        train_re_0 += tmp_train_re_0
        train_re_1 += tmp_train_re_1
        train_re_2 += tmp_train_re_2
        nb_train_steps += 1
        
        loss.backward()  # Tính toán đạo hàm loss function theo lan truyền ngược
        torch.nn.utils.clip_grad_norm_(BERT_Model.parameters(), 1.0) # tránh việc bùng nổ gradient khi lan truyền ngược, giá trị tối đa của gradient là 1
        optimizer.step() #tối ưu gradient
        
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" precision score neg: {0:.4f}".format(train_pre_0/nb_train_steps))
    print(" precision score neu: {0:.4f}".format(train_pre_1/nb_train_steps))
    print(" precision score pos: {0:.4f}".format(train_pre_2/nb_train_steps))
    print(" recall score neg: {0:.4f}".format(train_re_0/nb_train_steps))
    print(" recall score neu: {0:.4f}".format(train_re_1/nb_train_steps))
    print(" recall score pos: {0:.4f}".format(train_re_2/nb_train_steps))
    print(" F1 score neg: {0:.4f}".format(train_f1_0/nb_train_steps))
    print(" F1 score neu: {0:.4f}".format(train_f1_1/nb_train_steps))
    print(" F1 score pos: {0:.4f}".format(train_f1_2/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))
    print("Running Validation...")
    #Sử dụng đánh giá mô hình
    BERT_Model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    eval_f1_0 = 0
    eval_f1_1 = 0
    eval_f1_2 = 0
    eval_pre_0 = 0
    eval_pre_1 = 0
    eval_pre_2 = 0
    eval_re_0 = 0
    eval_re_1 = 0
    eval_re_2 = 0
    for batch in tqdm(val_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad(): #không update tham số
            outputs = BERT_Model(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1, tmp_eval_f1_0, tmp_eval_f1_1, tmp_eval_f1_2, tmp_eval_pre_0, tmp_eval_pre_1, tmp_eval_pre_2, tmp_eval_re_0, tmp_eval_re_1, tmp_eval_re_2 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            eval_f1_0 += tmp_eval_f1_0
            eval_f1_1 += tmp_eval_f1_1
            eval_f1_2 += tmp_eval_f1_2
            eval_pre_0 += tmp_eval_pre_0
            eval_pre_1 += tmp_eval_pre_1
            eval_pre_2 += tmp_eval_pre_2
            eval_re_0 += tmp_eval_re_0
            eval_re_1 += tmp_eval_re_1
            eval_re_2 += tmp_eval_re_2
            nb_eval_steps += 1
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
    print(" precision score neg: {0:.4f}".format(eval_pre_0/nb_eval_steps))
    print(" precision score neu: {0:.4f}".format(eval_pre_1/nb_eval_steps))
    print(" precision score pos: {0:.4f}".format(eval_pre_2/nb_eval_steps))
    print(" recall score neg: {0:.4f}".format(eval_re_0/nb_eval_steps))
    print(" recall score neu: {0:.4f}".format(eval_re_1/nb_eval_steps))
    print(" recall score pos: {0:.4f}".format(eval_re_2/nb_eval_steps))
    print(" F1 score neg: {0:.4f}".format(eval_f1_0/nb_eval_steps))
    print(" F1 score neu: {0:.4f}".format(eval_f1_1/nb_eval_steps))
    print(" F1 score pos: {0:.4f}".format(eval_f1_2/nb_eval_steps))
print("Training complete!")

======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0it [00:00, ?it/s]

 Accuracy: 0.8315
 F1 score: 0.8350
 precision score neg: 0.8471
 precision score neu: 0.5652
 precision score pos: 0.9272
 recall score neg: 0.8327
 recall score neu: 0.6598
 recall score pos: 0.8995
 F1 score neg: 0.8319
 F1 score neu: 0.5856
 F1 score pos: 0.9092
 Average training loss: 0.4362
Running Validation...


  0%|          | 0/23 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

 Accuracy: 0.8678
 F1 score: 0.8698
 precision score neg: 0.8762
 precision score neu: 0.6658
 precision score pos: 0.9415
 recall score neg: 0.8685
 recall score neu: 0.7203
 recall score pos: 0.9181
 F1 score neg: 0.8677
 F1 score neu: 0.6742
 F1 score pos: 0.9267
 Average training loss: 0.3642
Running Validation...


  0%|          | 0/23 [00:00<?, ?it/s]

 Accuracy: 0.6984
 F1 score: 0.7094
 precision score neg: 0.8229
 precision score neu: 0.2758
 precision score pos: 0.7575
 recall score neg: 0.6575
 recall score neu: 0.4337
 recall score pos: 0.8188
 F1 score neg: 0.7250
 F1 score neu: 0.3179
 F1 score pos: 0.7835
======== Epoch 3 / 3 ========
Training...


0it [00:00, ?it/s]

In [ ]:
import pandas as pd
import seaborn as sns
metrics = (['Precision', 'Recall', 'f1-score'])
classes = ['Tiêu cực', 'Trung tính', 'Tích cực', 'accuracy', 'weighted avg', 'macro avg']
df = pd.DataFrame([[0.8334, 0.6948, 0.7435], [0.3041, 0.4359, 0.3233], [0.7889, 0.8093, 0.7886], [0.7181]*3, [0.7078, 0.6909, 0.678], [0.542, 0.5445, 0.5282]], classes, metrics)
df.head()

In [ ]:
sns.heatmap(pd.DataFrame(df).iloc[:-1, :], annot=True)